In [20]:
# Import the required libraries
import hvplot.pandas
import pandas as pd


In [2]:
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.45,68,100,4.99,PF,2023-02-14 02:33:14
1,1,Saint-Philippe,-21.3585,55.7679,74.79,89,47,12.10,RE,2023-02-14 02:33:15
2,2,Jamestown,42.0970,-79.2353,35.24,83,0,11.50,US,2023-02-14 02:32:50
3,3,Yellowknife,62.4560,-114.3525,-15.45,76,75,9.22,CA,2023-02-14 02:33:16
4,4,Busselton,-33.6500,115.3333,75.72,46,0,14.03,AU,2023-02-14 02:33:16


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.45,68,100,4.99,PF,2023-02-14 02:33:14
4,4,Busselton,-33.6500,115.3333,75.72,46,0,14.03,AU,2023-02-14 02:33:16
11,11,Butaritari,3.0707,172.7902,81.70,70,100,14.34,KI,2023-02-14 02:33:20
15,15,Atuona,-9.8000,-139.0333,79.43,79,3,18.14,PF,2023-02-14 02:33:21
17,17,Ternate,0.8000,127.4000,81.00,83,100,16.58,ID,2023-02-14 02:33:22
19,19,Puerto Ayora,-0.7393,-90.3518,78.55,92,37,1.01,EC,2023-02-14 02:30:04
29,29,Bambous Virieux,-20.3428,57.7575,77.25,88,75,8.05,MU,2023-02-14 02:33:26
31,31,Kahului,20.8947,-156.4700,78.82,75,40,16.11,US,2023-02-14 02:33:27
36,36,Esperance,-33.8667,121.9000,86.49,39,64,6.82,AU,2023-02-14 02:33:29
37,37,Bengkulu,-3.8004,102.2655,82.13,67,74,5.95,ID,2023-02-14 02:33:29


In [11]:
preferred_cities_df.count()

City_ID       148
City          148
Lat           148
Lng           148
Max Temp      148
Humidity      148
Cloudiness    148
Wind Speed    148
Country       148
Date          148
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,79.45,-23.1203,-134.9692,
4,Busselton,AU,75.72,-33.6500,115.3333,
11,Butaritari,KI,81.70,3.0707,172.7902,
15,Atuona,PF,79.43,-9.8000,-139.0333,
17,Ternate,ID,81.00,0.8000,127.4000,
19,Puerto Ayora,EC,78.55,-0.7393,-90.3518,
29,Bambous Virieux,MU,77.25,-20.3428,57.7575,
31,Kahului,US,78.82,20.8947,-156.4700,
36,Esperance,AU,86.49,-33.8667,121.9000,
37,Bengkulu,ID,82.13,-3.8004,102.2655,


In [25]:
geoapify_key = "6218f3b10e6b4e30869dcf73a31f08a1"

In [28]:
import requests

In [29]:
# Set initial parameters to search for a hotel.
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

In [37]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Get the first hotel from the results and store the name
try:
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
    hotel_df.loc[index, "Hotel Name"] = "No hotel found"

In [36]:
# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City"
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [38]:
# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)